In [3]:
import geopandas as gpd
import pandas as pd

# Charger la BD Forêt
bd_foret_path = "../../data/project/FORMATION_VEGETALE.shp"  # Remplacez par le chemin vers votre fichier BD Forêt
bd_foret = gpd.read_file(bd_foret_path)

# Afficher les colonnes pour vérifier le bon nom de celles-ci
print("Colonnes disponibles dans la BD Forêt :", bd_foret.columns)

# Liste des classes à prendre en compte pour la classification à l'échelle du pixel
classes_incluses_pixel = [
    'Forêt fermée de feuillus purs',
    'Forêt fermée de résineux',
    'Forêt fermée de feuillus et résineux mélangés',
    'Forêt fermée de châtaignier pur'
]

# Liste des classes à prendre en compte pour la classification à l'échelle des peuplements
classes_incluses_peuplement = [
    'Forêt fermée de feuillus purs en îlots',
    'Forêt fermée de châtaignier pur',
    'Forêt fermée d’un autre feuillu pur'
]

# Appliquer la sélection des classes à l'échelle du pixel (en excluant les classes symbolisées par Ø)
bd_foret_pixel = bd_foret[bd_foret['TFV'].isin(classes_incluses_pixel)]

# Appliquer la sélection des classes à l'échelle des peuplements
bd_foret_peuplement = bd_foret[bd_foret['TFV'].isin(classes_incluses_peuplement)]

# Concaténer les deux DataFrames pour avoir toutes les classes pertinentes
bd_foret_final = pd.concat([bd_foret_pixel, bd_foret_peuplement])

# Sélectionner les colonnes requises : TFV, Code, et les attributs existants
# Si 'CODE' n'existe pas, vous devrez peut-être ajuster en fonction du nom exact de la colonne pour le code.
bd_foret_final = bd_foret_final[['TFV', 'CODE_TFV'] + [col for col in bd_foret.columns if col not in ['TFV', 'CODE_TFV']]]

# Enregistrer le fichier shapefile final
output_path = "../results/data/sample/Sample_BD_foret_T31TCJ.shp"
bd_foret_final.to_file(output_path)

# Affichage d'un message pour confirmer l'exécution
print(f"Le fichier shapefile a été sauvegardé à : {output_path}")


Colonnes disponibles dans la BD Forêt : Index(['ID', 'CODE_TFV', 'TFV', 'TFV_G11', 'ESSENCE', 'geometry'], dtype='object')
Le fichier shapefile a été sauvegardé à : ../results/data/sample/Sample_BD_foret_T31TCJ.shp
